### 将各个大洋插值到指定网格
#### use NCL(6.6.2)

grid_path掩膜原始格点，datapath目标网格格点

In [ ]:
output                 = "/home/yyq/mls/dataprocess/addwater/photo/grid"
outfile                = "/home/yyq/mls/dataprocess/nc/basin_new.nc"
grid_path              = "/home/yyq/mls/dataprocess/nc/BASIN_362X196_new.nc"
datapath               = "/data06/yyq/mls/licom2/addwater_new/exe/"

In [ ]:
f_grid                 = addfile(grid_path, "r")
f_year                 = addfile(datapath+"YME.nc", "r")

In [ ]:
lat_new                = f_year->lat
lon_new                = f_year->lon({0:359})  
printVarSummary(lat_new)
printVarSummary(lon_new)

In [ ]:
indd                   = (/f_grid->ind(:,{0:359})/)
lat                    = (/f_grid->lat/)
lon                    = (/f_grid->lon({0:359})/)  
printVarSummary(lat)
printVarSummary(lon)

整个大西洋赋值2，印度洋3，太平洋4

In [ ]:
indd                   = where(indd.eq.5,2,indd)
indd                   = where(indd.eq.7,4,indd)
indd                   = where(indd.eq.6,3,indd)

插值

In [ ]:
indd_new               = linint2 (lon,lat,indd, True, lon_new,lat_new, 0)

写入metadata

In [ ]:
indd_new!0             = "lat"                                                            
indd_new!1             = "lon"
lon_pts                = lon_new
lat_pts                = lat_new
lon_pts@units          = "degrees_east" 
lat_pts@units          = "degrees_north"
indd_new&lon           = lon_pts 
indd_new&lat           = lat_pts
printVarSummary(indd_new)

输出nc文件

In [ ]:
if (isfilepresent(outfile)) then
    system ("rm " + outfile)
end if
f_output = addfile(outfile, "c")
f_output->lat = lat_new
f_output->lon = lon_new
f_output->indd = indd_new

;画图检验

In [ ]:
wks      = gsn_open_wks("png", output)
plots    = new(1,"graphic")
res_0    = True
    res_0@gsnDraw                   =  False
    res_0@gsnFrame                  =  False
    res_0@cnFillOn                  =  True
    res_0@cnInfoLabelOn             =  False
    res_0@cnLinesOn                 =  False
    res_0@cnLineLabelsOn            =  False
    res_0@lbLabelBarOn              =  False
    res_0@mpCenterLonF              = 180
plots(0) = gsn_csm_contour_map(wks,indd_new, res_0)

resP    = True
    resP@gsnPanelLabelBar           = False
    resP@lbLabelAutoStride          = False
    resP@lbBoxSeparatorLinesOn      = False
    resP@lbBoxLinesOn               = False
    resP@lbBoxEndCapStyle           = "TriangleBothEnds"
gsn_panel(wks, plots, (/1,1/), resP)

![grid](../photo/grid.png)